In [1]:
# (CPU, DataSize, Y=Time)
import pandas as pd

In [2]:
df = pd.read_csv("testing_22July_dup.csv", sep=",") #data from 22July2016 
# copy the 'dup' folder to ~/ProfilingData/.
# remove *Output* files 
# run "python extract.py dup/" to generate testing.csv

In [3]:
len(df)

32

In [4]:
df.head(n=1)

ppn  mem seqlen  trial  AvgRSS  AvgSharedTxt  AvgStack  AvgTotal  \
0   11  64G    10M      1       0             0         0         0   

   AvgUnsharedData                                       CommandTimed  \
0                0  ./NGS-ann-program/BIN/cd-hit-dup -i ./data/no-...   

     ...     PageSize  PercentCPU  SignalsDeliv  SocketMsgRecv  SocketMsgSent  \
0    ...         4096         99%             0              0              0   

   Swaps  SystemTime  UserTime VolCtxtSwitch  WallTime  
0      0         3.5     42.32          1072   0:45.97  

[1 rows x 27 columns]

In [5]:
#find which columns are completely empty
a=len(df[df['AvgRSS'] > 0])
b=len(df[df['AvgSharedTxt'] > 0])
c=len(df[df['AvgStack'] > 0])
d=len(df[df['AvgTotal'] > 0])
e=len(df[df['AvgUnsharedData'] > 0])

print(a,b,c,d,e)

0 0 0 0 0


In [6]:
header = list(df.columns)

In [7]:
i = 1
a = 0
emptyColumns = []
for h in header:
    try:
        a=len(df[df[h] > 0])
        print(a,  h)
        if(a==0): #nothing is greater than zero for this column
            emptyColumns.append(h)
    except:
        print('......................'+ h +' not a scalar')
    i=i+1

32 ppn
......................mem not a scalar
......................seqlen not a scalar
32 trial
0 AvgRSS
0 AvgSharedTxt
0 AvgStack
0 AvgTotal
0 AvgUnsharedData
......................CommandTimed not a scalar
0 ExitStatus
32 FileSysIn
32 FileSysOut
32 InvolCtxtSwitch
32 MajorFaults
32 MaxRSS
32 MinorFaults
32 PageSize
......................PercentCPU not a scalar
0 SignalsDeliv
0 SocketMsgRecv
0 SocketMsgSent
0 Swaps
32 SystemTime
32 UserTime
32 VolCtxtSwitch
......................WallTime not a scalar


In [8]:
emptyColumns.append('trial')
emptyColumns.append('PageSize')
emptyColumns.append('CommandTimed')
emptyColumns.append('WallTime')

In [9]:
for c in emptyColumns:
    del df[c]

In [10]:
df.head(1)

ppn  mem seqlen  FileSysIn  FileSysOut  InvolCtxtSwitch  MajorFaults  \
0   11  64G    10M    3234924     2179426              212            3   

    MaxRSS  MinorFaults PercentCPU  SystemTime  UserTime  VolCtxtSwitch  
0  6544812      1388550        99%         3.5     42.32           1072

In [11]:
ft = df.copy(deep=True)

In [12]:
delthis = list(set(ft.columns.values) - set(['ppn','seqlen','SystemTime','UserTime']))
delthis

['MajorFaults',
 'PercentCPU',
 'MaxRSS',
 'mem',
 'FileSysIn',
 'InvolCtxtSwitch',
 'MinorFaults',
 'FileSysOut',
 'VolCtxtSwitch']

In [13]:
for c in delthis:
    del ft[c]

In [14]:
#DROP the lines which are NULL
#df = df[df.CommandTimed.notnull()]
len(df)

32

In [15]:
whatisthesize = {'5M': .54, '10M': 1.2,'15M': 1.7,'20M': 2.3,'25M': 2.9,'30M': 3.4,'35M': 4.0,'40M': 4.5}

In [16]:
ft['sizeGB'] = ft['seqlen'].map(whatisthesize)

In [17]:
del ft['seqlen']
ft.head(2)

ppn  SystemTime  UserTime  sizeGB
0   11        3.50     42.32     1.2
1   11        5.43     62.54     1.7

In [18]:
ft['Y_time']=ft['SystemTime']+ft['UserTime']
del ft['SystemTime']
del ft['UserTime']
ft.head()

ppn  sizeGB  Y_time
0   11     1.2   45.82
1   11     1.7   67.97
2   11     2.3   98.47
3   11     2.9  120.44
4   11     3.4  141.66

### Write to CSV

In [28]:
ft[['ppn','sizeGB','Y_time']].to_csv('22July_dup_ppn_sizeGB_Ytime.csv')

In [21]:
from sklearn.ensemble.forest import RandomForestRegressor
import numpy as  np
from sklearn.metrics import mean_squared_error

##### Generate Validation Set

In [24]:
from sklearn.cross_validation import train_test_split

#copy from ft_all
ft_all = ft.copy(deep=True)
ft0 =ft.copy(deep=True)

#shuffle
ft1 =ft0.iloc[np.random.permutation(len(ft0))]
ft2  =ft1.reset_index(drop=True)
ft2.head(2)

ppn  sizeGB  Y_time
0    8     4.5  203.98
1    2     4.0  166.80

In [25]:
#Generate Validation Set: split ft2

ft_subset, validationSet = train_test_split(ft2, test_size = 0.1)
ft = ft_subset.copy(deep=True)

ft.shape, validationSet.shape, ft_all.shape
#both test and train are taken from ft

((28, 3), (4, 3), (32, 3))

In [26]:
rmse = 10000
while(rmse > 50):
    np.random.seed()
    ft['is_train']=np.random.uniform(0,1,len(ft)) <= .8
    train, test = ft[ft['is_train']==True], ft[ft['is_train']==False]
    del ft['is_train']
    len(train)*1.0 / (len(test)+len(train))
    features = ft.columns[0:2]
    target = ft.columns[2]
    X_train = train[features]
    Y_train = train[target]
    X_test = test[features]
    Y_test = test[target].as_matrix()
    model = RandomForestRegressor(n_estimators=1500, n_jobs=-1)
    model.fit(X_train, Y_train.values)
    Y_prediction = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, Y_prediction))


In [27]:
X_validation = validationSet[features]
Y_validation = validationSet[target]

Y_prediction = model.predict(X_validation)
rmse_validation = np.sqrt(mean_squared_error(Y_validation, Y_prediction))

rmse_validation

3.0170355348330737

In [32]:
Y_validation.values #Print Y values of test dataset

array([ 121.17,   45.97,  166.8 ,  204.8 ])

In [33]:
Y_prediction #Print Y predicted values of test dataset

array([ 121.38728667,   47.87312667,  169.93840667,  200.01551333])

In [35]:
from sklearn import cross_validation
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='mean_squared_error')

In [36]:
scores

array([-27.62851778,  -2.17148238, -18.17896665, -41.15069196,
       -15.20797846, -13.86889157, -12.09585698, -51.42212725,
        -1.03207831,  -1.85403823])

In [75]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -8.16 (+/- 15.72)


In [76]:
ft[target].mean(), ft[target].std() * 2

(108.51031250000001, 118.18468799728724)

In [77]:
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='r2')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.99 (+/- 0.01)
